# First Data Processing Step
### $Time$ $Series$ $3rd$ $Test$

$Vasco$ $Mergulhão$ $-$ $Jan$ $2023$

### Version 1:
 - Imports 2 Raw Datasets:
     - Full Credit History.
     - Customer Information.
 - Filters valid IDs based on:
     - Customer type criteria.
     - Usable credit history records.
 - Creates Sub-samples:
     - Random 3x 10k IDs 
 - Outputs:
     - Reference ID list, with local IDs and Sampling Records.
     - Filtered Credit History.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import groupby
from datetime import timedelta, date
from pandas.core.common import flatten

import time
import datetime
import copy

# Raw Data Imports

## Credit History

In [2]:
Raw_CredHist = pd.read_csv('Data/Raw_Data/TS_FullSet_Credits_V2.csv')

In [3]:
Raw_CredHist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652046 entries, 0 to 191652045
Data columns (total 3 columns):
 #   Column             Dtype  
---  ------             -----  
 0   customer_id        object 
 1   date_timestamp     object 
 2   total_ontime_left  float64
dtypes: float64(1), object(2)
memory usage: 4.3+ GB


## Customer Info

In [4]:
Customer_Info = pd.read_csv('Data/Raw_Data/TS_CustomerInfo_V1.csv')

C:\Users\ucesvpm\AppData\Local\Temp\ipykernel_13056\4213000189.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  Customer_Info = pd.read_csv('Data/Raw_Data/TS_CustomerInfo_V1.csv')


In [5]:
Customer_Info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234134 entries, 0 to 234133
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   customer_id                     234134 non-null  object
 1   start_date                      234134 non-null  object
 2   gender                          234134 non-null  object
 3   customer_birth_date             233332 non-null  object
 4   preferred_language              234134 non-null  object
 5   main_source_income              234134 non-null  object
 6   main_source_income_other        929 non-null     object
 7   prev_light_energy_source        234134 non-null  object
 8   prev_light_energy_source_other  4105 non-null    object
 9   payg_converted                  233668 non-null  object
 10  country                         234121 non-null  object
 11  install_address_1               234120 non-null  object
 12  install_address_2             

---
# Valid ID Lists

## Customer Info

In [6]:
# SQL Criteria already filters most
# Only need to guarantee country not null, since sampling depends on it
validID_CustInfo = pd.DataFrame()
validID_CustInfo['customer_id'] = Customer_Info.customer_id.unique()
validID_CustInfo['country'] = Customer_Info['country']
validID_CustInfo = validID_CustInfo[~validID_CustInfo['country'].isnull()]
validID_CustInfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234121 entries, 0 to 234133
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  234121 non-null  object
 1   country      234121 non-null  object
dtypes: object(2)
memory usage: 5.4+ MB


## Credit History
- At least 180 days of valid data

In [7]:
validID_CredHist = pd.DataFrame()
validID_CredHist['customer_id'] = Raw_CredHist.customer_id.unique()
validID_CredHist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233922 entries, 0 to 233921
Data columns (total 1 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  233922 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB


In [8]:
validID_CredHist['record_length'] = Raw_CredHist.groupby('customer_id').size().values

In [10]:
validID_CredHist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233922 entries, 0 to 233921
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   customer_id    233922 non-null  object
 1   record_length  233922 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


### Minimum Record Length Filter

In [11]:
validID_CredHist = validID_CredHist[validID_CredHist['record_length']>=180]
validID_CredHist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219132 entries, 0 to 233920
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   customer_id    219132 non-null  object
 1   record_length  219132 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 5.0+ MB


## List Intersection

In [12]:
validID_Intersect = pd.DataFrame()
# Intersection ensures all IDs respect both sets of creteria
validID_Intersect ['customer_id']= list(set(validID_CustInfo['customer_id']).intersection(set(validID_CredHist['customer_id'])))
validID_Intersect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219089 entries, 0 to 219088
Data columns (total 1 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  219089 non-null  object
dtypes: object(1)
memory usage: 1.7+ MB


In [13]:
validID_Intersect = validID_Intersect.merge(validID_CustInfo[['customer_id', 'country']], how = 'left', on = 'customer_id')
validID_Intersect

,customer_id,country
0,000274f21c484b0eb4960d7955599eac,Rwanda
1,e9b6a22319c3cd42fa3626f1c62c7bce,Rwanda
2,4d81dd9eac599f1c8217abaa943f1036,Rwanda
3,a9b40b68557bcaddb79868d99f99dcec,Rwanda
4,218e4514a303e0054b1cf52fcf7cc54d,Rwanda
...,...,...
219084,c7ce66ccbb4b645bb95fe34d02a34259,Rwanda
219085,11085e2972a1452bb30e9372637eb0f6,Kenya
219086,c0b65c39f9795e015173ebe2940d6c9c,Kenya
219087,2b906aeae02a498d65f341efa8687a22,Rwanda


---
# Sampling Implementation & Recording

 Split by counrty and do 3x 10k, non-intersecting.

In [14]:
def random_sampler (df_in, sample_size, n_samples, sample_name, country = None):
    df_out = df_in.copy(deep=True)
    df_out[sample_name] = 0
    #Filter by country
    if country:
        df_in = df_in[df_in['country'] == country]
    
    used_ids = []
    for s in range(1, n_samples + 1):
        #Sample within list of IDs still available
        sampled_ids = df_in[~df_in['customer_id'].isin(used_ids)].sample(n=sample_size, replace=False, random_state=42).customer_id.values
        #Record sampled IDs in sampling column
        df_out[sample_name] = np.where(df_out['customer_id'].isin(sampled_ids), s, df_out[sample_name])
        #Add sampled IDs to list of used IDs
        used_ids.append(sampled_ids)
        used_ids = list(flatten(used_ids))
           
    return df_out

In [15]:
ID_Sampling = validID_Intersect.copy(deep=True)

In [30]:
ID_Sampling = random_sampler (ID_Sampling, sample_size= 10000, n_samples = 3, sample_name = 'Rwanda_10k', country = 'Rwanda')

In [31]:
ID_Sampling

,customer_id,country,Kenya_10k,Rwanda_10k
0,000274f21c484b0eb4960d7955599eac,Rwanda,0,0
1,e9b6a22319c3cd42fa3626f1c62c7bce,Rwanda,0,0
2,4d81dd9eac599f1c8217abaa943f1036,Rwanda,0,0
3,a9b40b68557bcaddb79868d99f99dcec,Rwanda,0,0
4,218e4514a303e0054b1cf52fcf7cc54d,Rwanda,0,0
...,...,...,...,...
219084,c7ce66ccbb4b645bb95fe34d02a34259,Rwanda,0,0
219085,11085e2972a1452bb30e9372637eb0f6,Kenya,0,0
219086,c0b65c39f9795e015173ebe2940d6c9c,Kenya,0,0
219087,2b906aeae02a498d65f341efa8687a22,Rwanda,0,2


---
# Reference ID List

In [32]:
Ref_IDs = ID_Sampling.copy(deep=True)
Ref_IDs.drop(columns=['country'], inplace = True)

In [33]:
# Order of operations
# 1) Adding start_date columns as a reference for new short_ID order
# 2) Sort by start_date to ensure order, ignore index to create new IDs
# 3) Reset Index to use new index as the short_IDs
# 4) Drop start_date as it's no longer needed
Ref_IDs = Ref_IDs.merge(Customer_Info[['customer_id', 'start_date']], how= 'left', on= 'customer_id').sort_values(by=['start_date'], ignore_index=True).reset_index(names='short_ID').drop(columns=['start_date'])
# Setting 1st ID to 1
Ref_IDs['short_ID'] = Ref_IDs['short_ID'] + 1

In [34]:
Ref_IDs.head()

,short_ID,customer_id,Kenya_10k,Rwanda_10k
0,1,689aa5a01c216d8b16ed0250cebdc702,0,0
1,2,21a06929d23550339ee18d98b6e05fc2,0,2
2,3,ea8df9172e5ad7ce79b77bada66d3e3e,0,0
3,4,fe7e11989f267560ff76e3323b0250e0,0,2
4,5,c9514b2f4933a409cdc480511ef9137b,0,0


---
# Filtered Datasets

In [21]:
Filtered_CredHist = Raw_CredHist[Raw_CredHist['customer_id'].isin(Ref_IDs['customer_id'].unique())]

In [22]:
Filtered_CredHist = Filtered_CredHist.merge(Ref_IDs[['customer_id', 'short_ID']], how = 'left', on = 'customer_id')

In [23]:
Filtered_CredHist['date_timestamp'] =  pd.to_datetime(Filtered_CredHist['date_timestamp'])
Filtered_CredHist['date_timestamp'] = Filtered_CredHist['date_timestamp'].dt.date    
Filtered_CredHist.rename(columns={"date_timestamp": "date"}, inplace = True)

In [24]:
Filtered_CredHist.sort_values(by = ['short_ID', 'date'], ignore_index=True, inplace=True)

In [25]:
Filtered_CredHist.drop(columns=['customer_id'], inplace=True)

In [26]:
cols = Filtered_CredHist.columns.tolist()
cols = cols[-1:] + cols[:-1]
Filtered_CredHist = Filtered_CredHist[cols]

In [27]:
Filtered_CredHist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179592709 entries, 0 to 179592708
Data columns (total 3 columns):
 #   Column             Dtype  
---  ------             -----  
 0   short_ID           int64  
 1   date               object 
 2   total_ontime_left  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.0+ GB


In [28]:
Raw_CredHist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652046 entries, 0 to 191652045
Data columns (total 3 columns):
 #   Column             Dtype  
---  ------             -----  
 0   customer_id        object 
 1   date_timestamp     object 
 2   total_ontime_left  float64
dtypes: float64(1), object(2)
memory usage: 4.3+ GB


---
# Saving Outputs

In [35]:
# Ref_IDs.to_csv('Data/FilteredIDs/Reference_ID_List.csv', index = False)

In [36]:
# Filtered_CredHist.to_csv('Data/FilteredIDs/TS_FilIDsSet_CredHist_V1.csv', index = False)